### 股票预测项目
本项目的目的是通过股票的历史行情价格来预测未来某只股票的涨跌。 问题本身其实就是二分类问题。数据是通过```tushare```库来获取到的，在压缩包里已经给出了一只股票的数据。本作业的目的是：
1. 根据已经给定的数据，构造出样本数据。在样本数据的构造过程我们需要使用特征工程，这个特征工程其实就是技术指标的提取。 
2. 提取完技术指标之后，做一些简单的数据处理
3. 构造训练数据和测试数据
4. 利用随机森立学习二分类器

本项目的重点是技术指标的提取，但为了方便大家，这些指标已经写好，建议可以去看一下每一个技术指标是如何定义的。

预估项目完成时间： 2小时

In [1]:
# 导入相应的函数库
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# 技术指标的提取函数
from technical_indicators import *

In [2]:
# 导入股票数据，下面的股票数据是通过tushare库来获得的
stock = pd.read_csv("600519.csv")
stock.head()

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
0,2019-09-12,1066.00,1109.98,1099.00,1066.00,41211.33,29.48,2.76,1114.276,1126.115,1108.345,40942.17,37385.21,37563.02
1,2019-09-11,1119.22,1119.97,1069.52,1068.00,81716.54,-54.33,-4.83,1123.276,1127.525,1105.640,39286.08,36197.12,36864.33
2,2019-09-10,1134.30,1135.00,1123.85,1120.01,26227.07,-12.67,-1.11,1134.374,1130.584,1104.330,29662.81,32726.43,34849.05
3,2019-09-09,1145.00,1148.00,1136.52,1135.00,29379.34,-5.97,-0.52,1137.604,1129.099,1099.035,30314.42,35320.15,35054.18
4,2019-09-06,1144.50,1146.15,1142.49,1131.00,26176.59,-1.51,-0.13,1138.052,1125.742,1093.141,30085.41,37232.00,37660.16


### 1. 对于股票数据提取技术指标
直接调用给定的技术指标库来获得这些数据， 但建议大家可以简单看一下这些指标是如何被计算出来的。 虽然没必要一定要掌握，但大致的计算逻辑可以学习一下的。 如果对某一种指标感兴趣，想深入理解建议在百度上搜索 ： “技术指标” + “指标名字”来获得相关的参考资料，比如搜索 “技术指标” + 'rate of change"， 有大量的资料可以参考的。

> ```TODO1```: 提取技术指标

In [3]:
# TODO: 提取各类技术指标， 你可以把所有的技术指标全部调用一遍，也可以选择几个来尝试。 或者感兴趣的话，可以把其他的技术指标也加进来。 
#       每个技术指标的参数是不一样的，但基本也就1-2个参数，最常用的参数是天数（函数里用n来表示）， 有些技术指标需要传入两个参数（比如MACD，
#       一个是针对于fast_line, 一个是针对于slow_line, 需要分别定义天数）。 由于每个指标都有参数，所以针对于同一类指标其实可以提取很多不同的特征的！

# 例子： stock = average_directional_movement_index(stock, 12, 26) #  提取技术指标并存放在新的dataframe中
#      stock = moving_average(stock, 5)
#      stock = moving_average(stock, 15)

#stock = moving_average(stock, 5)
stock = exponential_moving_average(stock, 5)
stock = momentum(stock, 10)
stock = rate_of_change(stock, 5)
stock = average_true_range(stock, 14)
stock = bollinger_bands(stock, 20)
stock = stochastic_oscillator_k(stock)
stock = stochastic_oscillator_d(stock, 14)
stock = macd(stock, 12, 26)
stock = ease_of_movement(stock, 14)
stock = standard_deviation(stock, 14)

### 2. 数据处理，以及训练样本和测试样本的提取
通过上面的环节我们已经提取好了所需要的技术指标。 接下来的环节是通过这批数据来构造训练数据和测试数据了。 具体构造用于监督学习的数据的方法在本章的视频课程里已经提过，可以按照此方法来做。 
注：数据中存在着NAN， 稍微思考一下为什么会出现这些NAN？ 为了去理解这些NAN的源头，需要看一下pandas里的rolling().mean()是如何工作的。 在我们项目中，我们是通过历史一段时间的数据来预测未来的涨跌的，所以一定不能使用未来数据来预测未来，只能用历史数据来预测未来。 

> ```TODO2```： 做必要的数据预处理，并构建好样本数据。这里我们要预测的标签是第二天的涨跌。如果第二天的```close```价格 >  第一天的```close```价格，我们可以认为这个样本为正样本（1）， 如果价格小于第一天的```close```价格，就认为这个样本为负样本（0）。 构建完训练样本之后，在把样本通过```train_test_split```来划分为训练集和测试集。

In [4]:
# date column feature engineering
stock['date'] = pd.to_datetime(stock['date'])

stock['year'] = stock.date.apply(lambda x: x.year)
stock['month'] = stock.date.apply(lambda x: x.month)
stock['weekday'] = stock.date.apply(lambda x: x.weekday())

In [5]:
# TODO 2   完成样本数据的构造，并随机分成训练和测试数据

# generate tag: whether price goes up next day, if yes: 1
tag = pd.Series([1 if stock.close[i+1] > stock.close[i] else 0 for i in range(len(stock)-1)], name = 'tag')
stock = stock.join(tag)
# remove NAN rows
stock = stock.dropna()

features = [col for col in stock.columns if ('tag' not in col and 'date' not in col)]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    stock[features].values, 
    stock['tag'].values,
    test_size = 0.2,
    random_state = 10)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(463, 29) (463,) (116, 29) (116,)


### 3. 利用随机森林训练模型
模型训练部分跟之前没有太大区别，试着通过交叉验证来训练一下，然后看看结果如何。 
> ```TODO3```：训练模型 

In [6]:
# TODO: 训练随机森林模型，请尝试不同的参数，最后在测试集上输出最好的参数
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

params_n_estimators = np.linspace(10, 100, 4, dtype = int)
params_min_samples_split = np.linspace(5, 20, 4, dtype = int)
params_min_samples_leaf = np.linspace(2, 10, 5, dtype = int)
params_max_depth = np.linspace(4, 10, 4, dtype = int)
params_min_weight_fraction_leaf = np.array([0.01, 0.02])
 
rf = RandomForestClassifier(criterion = 'entropy')
parameters = {'n_estimators': params_n_estimators,
              'min_samples_split': params_min_samples_split,
              'min_samples_leaf': params_min_samples_leaf,
              'max_depth': params_max_depth,
              'min_weight_fraction_leaf': params_min_weight_fraction_leaf}

model = GridSearchCV(rf, parameters, scoring = 'f1', cv = 5)
model.fit(X_train, y_train)

# 输出最好的参数 
print(model.best_params_)

{'max_depth': 4, 'min_samples_leaf': 6, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.02, 'n_estimators': 10}


In [7]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        53
         1.0       1.00      1.00      1.00        63

    accuracy                           1.00       116
   macro avg       1.00      1.00      1.00       116
weighted avg       1.00      1.00      1.00       116



In [15]:
from sklearn.metrics import confusion_matrix
#train = model.predict(X_train)
confusion_matrix(y_test, predictions)

array([[53,  0],
       [ 0, 63]])

> ```TODO4```: 问答题：得出来的结果怎么样？ 是否满足预期？ 你觉得有什么方式可以提升模型的准确率？ 

In [ ]:
问答题回复：模型在测试数据集表现完美（好于训练数据集），满足预期。提升模型准确率的方法：Feature Engineering，Hyperparameter Tuning

拓展阅读： 从本项目中可以看到这里的核心其实就是一个一个指标，而且每一个指标都是通过大量的经验来构造出来的。 但有些复杂度的指标确实也比较难想出来。问题：有没有可能让计算机学出有用的指标呢？ 比如计算机可以学出这样的指标 = (close- open) * volum - close * close - open   虽然这个指标有点看不懂，但有可能是有效的，有没有可能让AI做这件事情？？？ 如果对这些感兴趣，可以参考一下下方链接： https://www.baidu.com/link?url=WmpaRS35js8T8gAUzaF6_rvdepe0OqpgmeU0fTxhXzMZnKCUXIECQeUFB6VTpFjg&wd=&eqid=b04a03b600117ba2000000035d88bea9